# Research on Loan Repayment Reliability

Here I import the pandas library and read the data from the csv file.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

Then I display the first 20 lines of the dataframe `data` on the screen

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,married,0,F,employee,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,married,0,F,employee,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,married,0,M,employee,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,married,0,M,employee,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,civil,1,F,pensioner,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,civil,1,M,partner,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,married,0,F,partner,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,married,0,M,employee,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,civil,1,F,employee,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,married,0,M,employee,0,144425.938277,покупка жилья для семьи


After that, I display the general information about the dataframe `data` on the screen

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Data preprocessing

### Processing missing values

I will check the dataframe `data` for missing values using the `isna()` method and display the number of missing values for each column using the `sum()` method.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in two columns. One of them is `days_employed`. I'll handle the missing values in this column in the next step. The other column with missing values is `total_income`, which contains income data. The income amount is most influenced by the employment type, so the missing values in this column should be filled with the median value for each type from the `income_type` column.

In [6]:
def fill_missing_income(row):
    if pd.isnull(row['total_income']):
        median_income = data.groupby('income_type')['total_income'].median()[row['income_type']]
        return median_income
    else:
        return row['total_income']

data['total_income'] = data.apply(fill_missing_income, axis=1)

### Processing anomalies

The data may contain artifacts (anomalies) – values that do not reflect reality and appeared due to some error. One such artifact is a negative number of days of employment in the `days_employed` column. I will process the values in this column by replacing all negative values with positive ones using the `abs()` method.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Here, for each employment type, display the median value of days_employed in days.

In [8]:
median_days_employed = data.groupby('income_type')['days_employed'].median()
print(median_days_employed)

income_type
civil_servant           2689.368353
employee                1574.202821
entrepreneur             520.848083
on_maternity_leave      3296.759962
partner                 1547.382223
pensioner             365213.306266
student                  578.751554
unemployed            366413.652744
Name: days_employed, dtype: float64


We can see that for two types (unemployed and pensioners) the values will be abnormally large.

Display the unique values of the `children` column.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

There are two anomalies in the `children` column: -1 and 20. I will delete them

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

To check the result, I will display the unique values of the `children` column again.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Processing duplicates

I will fill the missing values in the `days_employed` column with the median values for each employment type from the `income_type` column.

In [12]:
def fill_missing_days(row):
    if pd.isnull(row['days_employed']):
        median_income = data.groupby('income_type')['days_employed'].median()[row['income_type']]
        return median_income
    else:
        return row['days_employed']
    
data['days_employed'] = data.apply(fill_missing_days, axis=1)

Make sure all missing values have been filled. I will verify this by displaying the number of missing values for each column using two methods.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Changing data type

Now, I will convert the `total_income` column from a float data type to an integer using the `astype()` method.

In [14]:
data['total_income'] = data['total_income'].astype('int')

### Processing duplicates

Here, I will handle the implicit duplicates in the `education` column. There are identical values recorded differently using uppercase and lowercase letters and I will convert all values to lowercase.

In [15]:
data['education'] = data['education'].str.lower()

Display the number of duplicate rows in the data. If such rows are present, remove them.

In [16]:
print(data.duplicated().sum())

71


In [17]:
data = data.drop_duplicates(keep='first')

### Data categorization

Based on the ranges specified below, I will create a column `total_income_category` in the data dataframe with the following categories:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(row):
    total_income = row['total_income']
    if total_income <= 30000:
        return 'E'
    elif total_income <= 50000:
        return 'D'
    elif total_income <= 200000:
        return 'C'
    elif total_income <= 1000000:
        return 'B'
    else:
        return 'A'

In [19]:
data['total_income_category'] = data.apply(categorize_income, axis=1)

Display the list of unique loan purposes from the purpose column.   

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об


Now, I will create a function that, based on the data from the purpose column, will form a new column purpose_category with the following categories:

- `'operations with a car'`,
- `'real estate operations'`,
- `'wedding planning'`,
- `'getting an education'`.

In [21]:
def categorize_purpose(row):
    purpose = row['purpose']
    if 'автомобил' in purpose:
        return 'operations with a car'
    elif 'недвижимост' in purpose or 'жиль' in purpose:
        return 'real estate operations'
    elif 'свадьб' in purpose:
        return 'wedding planning'
    else:
        return 'getting an education'

In [22]:
data['purpose_category'] = data.apply(categorize_purpose, axis=1)

#### Is there a relationship between the number of children and timely loan repayment?

In [23]:
pivot_table = pd.pivot_table(data, index=['children'], values=['debt'], aggfunc=['sum', 'count', 'mean']).reset_index()
pivot_table[('mean', 'debt')] = pivot_table[('mean', 'debt')] * 100
pivot_table

,children,sum,count,mean
,,debt,debt,debt
0,0,1063,14091,7.543822
1,1,444,4808,9.234609
2,2,194,2052,9.454191
3,3,27,330,8.181818
4,4,4,41,9.756098
5,5,0,9,0.000000


Based on the table, it is evident that people without children are the most creditworthy, with the share of debtors among them being 7.5%. People with 1 or 2 children are less likely to repay the loan, with their non-repayment rate being 9.2% and 9.4%, respectively. The remaining groups cannot be analyzed correctly as their sample size is too small.

#### Is there a relationship between marital status and timely loan repayment?

In [24]:
pivot_table = pd.pivot_table(data, index=['family_status'], values=['debt'], aggfunc=['sum', 'count', 'mean']).reset_index()
pivot_table[('mean', 'debt')] = pivot_table[('mean', 'debt')] * 100
pivot_table

,family_status,sum,count,mean
,,debt,debt,debt
0,civil,385,4134,9.313014
1,divorced,84,1189,7.064760
2,married,927,12261,7.560558
3,single,273,2796,9.763948
4,widow,63,951,6.624606


Analyzing the table, we can conclude that widowed individuals are the most reliable in repaying loans, with a debtor rate of 6.6%. They are followed by divorced individuals, with a rate of 7%. Married people and those in a civil partnership have lower creditworthiness, with rates of 7.5% and 9.3%, respectively. The least reliable in repaying loans are single individuals, with a debtor rate of 9.7%.

#### Is there a relationship between income level and timely loan repayment?

In [25]:
pivot_table = pd.pivot_table(data, index=['total_income_category'], values=['debt'], aggfunc=['sum', 'count', 'mean']).reset_index()
pivot_table[('mean', 'debt')] = pivot_table[('mean', 'debt')] * 100
pivot_table

,total_income_category,sum,count,mean
,,debt,debt,debt
0,A,2,25,8.000000
1,B,354,5014,7.060231
2,C,1353,15921,8.498210
3,D,21,349,6.017192
4,E,2,22,9.090909


Among all income categories, only two can be analyzed – B and C, as the sample size for the others is quite small. Among people with income type B, 7% did not repay their loans; their income ranges from 200,001 to 1,000,000. Among those with income type C (50,001–200,000), 8.5% did not repay their loans. These proportions make sense, as a higher income level means a greater likelihood of loan repayment, while those in the low and high-income categories either do not receive/take loans or do not need them.

#### How do different loan purposes affect timely repayment?

In [26]:
pivot_table = pd.pivot_table(data, index=['purpose_category'], values=['debt'], aggfunc=['sum', 'count', 'mean']).reset_index()
pivot_table[('mean', 'debt')] = pivot_table[('mean', 'debt')] * 100
pivot_table

,purpose_category,sum,count,mean
,,debt,debt,debt
0,getting an education,369,3988,9.252758
1,operations with a car,400,4279,9.347978
2,real estate operations,780,10751,7.255139
3,wedding planning,183,2313,7.911803


In this table, we can see that people who take out loans for real estate or for a wedding are more likely to repay them, with debtor rates of 7.2% and 7.9%, respectively. Loans for education and car-related operations are less likely to be repaid, with debtor rates of 9.25% and 9.34%, respectively.

#### Possible reasons for missing values in the original data.


Most likely, this is due to human factors. It's also possible that the `total_income` and `days_employed` columns were incorrectly recorded by the system, as they might have been optional in the form, or the person was unemployed. It can be noted that the number of missing cells in both columns is 2,174, which likely indicates that errors were made at some stage of data collection by the system.

#### Why filling in missing values with the median is the best solution for quantitative variables?

This approach is more representative because the mean can be significantly 'skewed' by individuals who earn excessively high amounts or have an unusually large number of workdays.

### Conclusion

After data preprocessing (removing missing values, handling anomalous values, dealing with duplicates, and categorizing data), certain conclusions can be drawn from the study.

Firstly, **people without children are the best at repaying loans**, with the share of debtors in their category being 7.5%. They are followed by people with 1 or 2 children, with shares of 9.2% and 9.4%, respectively. The sample size of people with fewer children was too small to draw any conclusions.

Secondly, marital status can also affect loan repayment. **Widowed individuals are most likely to repay loans**, with their debtor rate being 6.6%. They are followed by divorced individuals with a rate of 7%. Married people and those in a civil partnership have lower creditworthiness, with rates of 7.5% and 9.3%, respectively.

Thirdly, income can have a direct impact on whether a loan will be repaid or not. Based on the analysis, it can be concluded that people with extremely high or low incomes are less likely to take out a loan (as indicated by the low sample size). People with an income from 50,001 to 200,000 have a debtor rate of 8.5%, while those with an income from 200,000 to 1,000,000 have a rate of 7%. These rates are logical – **the higher the income, the greater the client's creditworthiness.**

Finally, the purpose of the loan can also be a contributing factor in "predicting" loan repayment. Loans for real estate and weddings have debtor rates of 7.2% and 7.9%, respectively. Loans for education and car-related operations have higher debtor rates, with shares of 9.25% and 9.34%, respectively.

Thus, people without children or with 1-2 children, as well as those with an income ranging from 200,001 to 1,000,000, have a better credit history, and widowed or divorced individuals are more likely to repay loans on time. Loans for real estate or weddings are most likely to be repaid, while loans for education or a car are associated with a higher probability of delinquency.

### **Ideal Client Profile:**
- **Children:** No children or has 1-2 children.
- **Income:** Has an above average income
- **Marital Status:** Widowed or divorced.
- **Loan Purpose:** Takes loans for real estate or wedding purposes.

This profile suggests a higher likelihood of repaying loans on time, making them a desirable and reliable client for lenders.

### **Undesirable Client Profile:**
- **Children:** Likely has more than 2 children.
- **Income:** Has a below average income.
- **Marital Status:** Single, in a civil partnership, or married
- **Loan Purpose:** Takes loans for education or car-related operations.

This profile indicates a higher risk of not repaying loans on time, making them less desirable for lenders.
